In [2]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [3]:
SEAsia_extent = ee.Image("users/ruarai/SEAsia_extent")

In [4]:
cats_interest = {
    'open_shrublands': 7,
    'woody_savannas': 8,
    'savannas': 9,
    'grasslands': 10,
    'permanent_wetlands': 11,
    'croplands': 12,
    'cropland_natural_vegetation_mosaic': 14
}

In [182]:
sea_projection = SEAsia_extent.projection()


im_geo = SEAsia_extent.geometry()

sea_projection.nominalScale().getInfo()



4638.310261061552

In [253]:
dataset = ee.ImageCollection('MODIS/006/MCD12Q1')
land_cover = dataset.select('LC_Type1')

land_cover_year = ee.Image(land_cover.toList(100).get(-7))


#land_cover_year = land_cover_year.reproject(crs = ee.Projection('EPSG:3857'),
#                                            scale = sea_projection.nominalScale().divide(10))

maps_by_cat = {}

for name, value in cats_interest.items():
    map_of_cat = land_cover_year.eq(value)
    map_of_cat = map_of_cat.toDouble()
    
    map_of_cat = map_of_cat.reduceResolution(reducer = ee.Reducer.sum().unweighted(),
                                             maxPixels = 1000)

    map_of_cat = map_of_cat.reproject(crs = sea_projection)

    map_of_cat = map_of_cat.clip(im_geo)
    
    map_of_cat = map_of_cat.updateMask(SEAsia_extent.eq(0))
    
    maps_by_cat[name] = map_of_cat
    
maps_by_cat

{'open_shrublands': <ee.image.Image at 0x11318ee71f0>,
 'woody_savannas': <ee.image.Image at 0x11318ee76a0>,
 'savannas': <ee.image.Image at 0x11318ee7b20>,
 'grasslands': <ee.image.Image at 0x11318ef0070>,
 'permanent_wetlands': <ee.image.Image at 0x11318ef04f0>,
 'croplands': <ee.image.Image at 0x11318ef0970>,
 'cropland_natural_vegetation_mosaic': <ee.image.Image at 0x11318efa760>}

In [246]:
maps_collected = ee.Image(list(maps_by_cat.values()))
maps_collected = maps_collected.toDouble()
maps_collected = maps_collected.rename(list(maps_by_cat.keys()))

In [254]:
import folium


vis_bands = {
  'bands': ['croplands', 'grasslands', 'woody_savannas'],
  'min': 0.0,
  'max':100.0,
  'gamma': [0.95, 1.1, 1]
};
vis = {
  'min': 0.0,
  'max':100.0,
};


map_test = folium.Map(location=[-5,110], zoom_start=6)


map_test.add_ee_layer(maps_by_cat['woody_savannas'], vis, 'test')

display(map_test)

In [121]:
task = ee.batch.Export.image.toDrive(image=maps_collected,
                                     description='SEA_landcover_mcd12q1_uw',
                                     crs='EPSG:4326',
                                     scale=sea_projection.nominalScale(),
                                     fileFormat='GeoTIFF')
task.start()

In [133]:
task.status()

{'state': 'COMPLETED',
 'description': 'SEA_landcover_mcd12q1_uw',
 'creation_timestamp_ms': 1617000126570,
 'update_timestamp_ms': 1617000533586,
 'start_timestamp_ms': 1617000144119,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'id': 'HVEGESASWI67ZM2VZSHSWC3V',
 'name': 'projects/earthengine-legacy/operations/HVEGESASWI67ZM2VZSHSWC3V'}